In [1]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.1 MB/s eta 0:00:00


In [86]:
import mne
import numpy as np
import scipy.io as sio

from mne.decoding import CSP
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

###############################################################################
# 1) Load the training GDF
train_gdf = "A07T.gdf"  # e.g., for subject 8
raw_train = mne.io.read_raw_gdf(train_gdf, preload=True)

# Filter the data (7-30 Hz)
raw_train.filter(7.0, 30.0, fir_design='firwin')

###############################################################################
# 2) Load the corresponding .mat file
#    This file typically has something like "classlabel" of shape (288,)
train_mat = "A07T.mat"
mat_train = sio.loadmat(train_mat)
# Suppose the variable is named 'classlabel'
true_labels_train = mat_train["classlabel"].ravel()  # shape (288,)

print("Train labels shape:", true_labels_train.shape)

###############################################################################
# 3) Get events for trial onsets from the GDF
#    Typically '768' marks the start of each trial
events_train, event_id_train = mne.events_from_annotations(
    raw_train, event_id={"768": 1}
)
# Now MNE sees all "768" annotations as event code = 1
print("events_train.shape:", events_train.shape)  # hopefully (288, 3)

###############################################################################
# 4) Create epochs (one epoch per trial)
tmin, tmax = 0.0, 4.0
epochs_train = mne.Epochs(
    raw_train,
    events_train,
    event_id={"trial": 1},
    tmin=tmin,
    tmax=tmax,
    picks="eeg",
    baseline=None,
    preload=True
)

X_train_full = epochs_train.get_data()  # shape (n_trials, n_channels, n_times)
print("X_train_full.shape:", X_train_full.shape)

###############################################################################
# 5) Filter to just classes 1=left, 2=right (if you're doing 2-class)
is_left_or_right = np.isin(true_labels_train, [1, 2])
X_train = X_train_full[is_left_or_right]
y_train = true_labels_train[is_left_or_right]

# Map 1->0, 2->1 for binary classification
y_train[y_train == 1] = 0
y_train[y_train == 2] = 1

print("Final shapes after class filtering:", X_train.shape, y_train.shape)

###############################################################################
# 6) Create your CSP + SVM pipeline and train
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
svm = SVC(kernel="linear")

pipeline = Pipeline([("CSP", csp), ("SVM", svm)])

# (Optional) internal train/val split
X_sub, X_val, y_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)
pipeline.fit(X_sub, y_sub)

val_preds = pipeline.predict(X_val)
val_acc = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy on training split: {val_acc:.2f}")

# Retrain on all training data if desired
pipeline.fit(X_train, y_train)


Extracting EDF parameters from /content/A07T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 681070  =      0.000 ...  2724.280 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.8s


Train labels shape: (288,)
Used Annotations descriptions: ['768']
events_train.shape: (288, 3)
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 original time points ...
0 bad epochs dropped
X_train_full.shape: (288, 25, 1001)
Final shapes after class filtering: (144, 25, 1001) (144,)
Computing rank from data with rank=None
    Using tolerance 4.1e-05 (2.2e-16 eps * 25 dim * 7.4e+09  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Validation Accuracy on training split: 0.66
Computing rank from data with rank=None
    Using tolerance 4.6e-05 (2.2e-16 eps * 25 dim * 8.4e+09  max singular value)
    Estimated rank (data): 25
    data: rank 25 computed from 25 data channels with

Pipeline(steps=[('CSP', CSP(log=True)), ('SVM', SVC(kernel='linear'))])

In [87]:
import mne
import numpy as np
import scipy.io as sio
from sklearn.metrics import accuracy_score

# Reuse the same pipeline object if it's in scope,
# or re-load from disk if you saved it, etc.

###############################################################################
# 1) Load the evaluation GDF
test_gdf = "A07E.gdf"
raw_eval = mne.io.read_raw_gdf(test_gdf, preload=True)

# Same filter
raw_eval.filter(7.0, 30.0, fir_design='firwin')

###############################################################################
# 2) Load the .mat file for evaluation
test_mat = "A07E.mat"
mat_test = sio.loadmat(test_mat)
true_labels_test = mat_test["classlabel"].ravel()  # shape (288,) typically
print("Test labels shape:", true_labels_test.shape)

###############################################################################
# 3) Get events from GDF for trial start
events_eval, event_id_eval = mne.events_from_annotations(
    raw_eval, event_id={"768": 1}
)

###############################################################################
# 4) Create epochs
epochs_eval = mne.Epochs(
    raw_eval,
    events_eval,
    event_id={"trial": 1},
    tmin=0.0,
    tmax=4.0,
    picks="eeg",
    baseline=None,
    preload=True
)
X_eval_full = epochs_eval.get_data()
print("X_eval_full.shape:", X_eval_full.shape)

###############################################################################
# 5) Filter for classes 1=left, 2=right
is_left_or_right_test = np.isin(true_labels_test, [1, 2])
X_eval = X_eval_full[is_left_or_right_test]
y_eval = true_labels_test[is_left_or_right_test]

# Map labels: 1->0, 2->1
y_eval[y_eval == 1] = 0
y_eval[y_eval == 2] = 1

print("X_eval.shape:", X_eval.shape, "y_eval.shape:", y_eval.shape)

###############################################################################
# 6) Predict and measure accuracy
preds = pipeline.predict(X_eval)
acc_test = accuracy_score(y_eval, preds)
print(f"Final Test Accuracy: {acc_test:.2f}")


Extracting EDF parameters from /content/A07E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 673134  =      0.000 ...  2692.536 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


ValueError: Onset, duration, description, and ch_names must be equal in sizes, got 5798007, 16745727, 0, and 5798007.